<a href="https://colab.research.google.com/github/Maraudier/tkfruit/blob/master/BF_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install kaggle

In [0]:
!mkdir .kaggle
!mkdir ~/.kaggle

In [0]:
import json
token = {"username":"williamahtou","key":"40faa6e3f4d012c9b39cf9e96a731583"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

In [0]:
!kaggle datasets download -d sriramr/fruits-fresh-and-rotten-for-classification -p /content

In [0]:
!unzip \*.zip

In [0]:
!rm -rf /content/dataset/dataset/

In [0]:
!ls

In [0]:
import pathlib
# Specifying test and training paths
train_dir = pathlib.Path('dataset/train')
test_dir = pathlib.Path('dataset/test')

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Version of tensorflow should be specified in Colab
%tensorflow_version 2.x
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
from tensorflow.keras import datasets, layers, models
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
image_count = len(list(train_dir.glob('*/*.png'))) + len(list(test_dir.glob('*/*.png')))
CLASS_NAMES = np.array([item.name for item in train_dir.glob('*') if item.name != 'LICENSE.txt'])
CLASS_NAMES

In [0]:
# The 1./255 is to convert from uint8 to float32 in range [0,1]. Split data into 80/20 
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# The transformations shear_range, zoom_range, horizontal_flip will improve accuracy across data and allow for more generalized inputs
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)
# CONSTANTS, currently unsure what to set IMG_HEIGHT and IMG_WIDTH
BATCH_SIZE = 64
IMG_HEIGHT = 128
IMG_WIDTH = 128
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [0]:

train_data_gen = train_generator.flow_from_directory(directory=str(train_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="sparse",
                                                     classes = list(CLASS_NAMES))
test_data_gen = test_generator.flow_from_directory(directory=str(test_dir),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="sparse",
                                                     classes = list(CLASS_NAMES))


In [0]:
# CNN architecure
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

In [0]:
model.summary()

In [0]:
# Setup checkpoint callback - will save the model after each epoch if validation loss is lower than the previous checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('./', save_best_only=True)

In [0]:
# Compiler
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training the data and validating with test data
history = model.fit(
        train_data_gen,
        epochs=25,
        callbacks=[checkpoint],
        validation_data=test_data_gen)


In [0]:

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.1, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_data_gen, verbose=2)

In [0]:
import base64

def save_model_signed(model, path):
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, ], dtype=tf.string)])
    def preprocess_and_evaluate(b64_img):
        img = tf.image.decode_image(b64_img[0], dtype=tf.uint8)
        img.set_shape((None, None, 3))
        img = tf.image.resize(img, [128, 128])
        img = tf.reshape(img, (-1, 128, 128, 3))
        img = tf.cast(img, dtype=tf.float32) / 255


        return model(img)
    
    tf.saved_model.save(model, path, signatures=preprocess_and_evaluate)

In [0]:
save_model_signed(model,'')